## Imports

In [1]:
############
# Built-in #
############
import ast
import os
import re
import tempfile

###############
# Third-party #
###############
import numpy as np
import pandas as pd
from sqlalchemy import create_engine

pd.set_option('display.max_columns', None)

## Database Connection Setup

In [2]:
HOST = 'localhost'
USER = 'db_user'
PORT = 5438
DATABASE = 'msd_db'
PASSWORD = 'LetMeIn'
TABLE_NAME = 'song_info'

# Connect with SQLAlchemy
conn_string = f'postgresql://{USER}:{PASSWORD}@{HOST}:{PORT}/{DATABASE}'
sqla_conn = create_engine(conn_string)

## Notebook Functions
Functions that are used throughout this notebook to either help load data or process data.

In [3]:
def read_sql_with_copy(query, db_engine, convert_list_cols=False):
    '''Run a SQL query using the COPY command to reduce RAM usage.
    
    By using a temporary file to run a query, this cuts down on the
    memory usage by utilizing a direct download to CSV rather than 
    dealing with all the overhead that pandas does internally with 
    its built-in `read_sql` function.
    
    Parameters
    ----------
    query : str
        Query string to pass into the database
        
    db_engine : SQLAlchemy engine object
        SQLAlchemy engine for creating connections
        
    convert_list_cols : bool
        If `True`, this will convert any columns that are supposed to be
        lists into actual lists instead of the strings that this method
        returns for those columns.
    '''
    print('Running query...', end = '')
    try:
        with tempfile.TemporaryFile() as tmp:
            copy_sql = f'COPY ({query}) TO STDOUT WITH CSV HEADER'
            conn = db_engine.raw_connection()
            cur = conn.cursor()
            cur.copy_expert(copy_sql, tmp)
            tmp.seek(0)
            df = pd.read_csv(tmp)
    except Exception as e:
        print(e)
    finally:
        cur.close()
        conn.close()
    print('done!')
    
    if convert_list_cols:
        # Get columns that are supposed to be lists
        print('Converting list columns...', end = '')
        list_cols = []
        for col in df.columns:
            try:
                # There is a song containing curly braces in the title so
                # this will pick up on that, hence the extra conditioning
                # on the 'title' column
                if any(df[col].str.contains('{')) and col != 'title': 
                    list_cols.append(col)
            except:
                # If a column doesn't have a `str` attribute, it is not a string
                # column so it should error out above and we can ignore that column
                continue

        # Change curly braces to brackets and change to list
        # using some funky regex here to properly wrap text
        # in double quotes so `ast.literal_eval` works, i.e.,
        # turns '{item_1, item_2, ...}' into '["item_1", "item_2", ...]'
        # so literal evaluation turns it into an actual list
        pattern = re.compile(r'{([\w\s,\-"&]+)}')
        f_inner = lambda x: '{{{}}}'.format(re.sub(r'[\w\s\-&]+', r'"\g<0>"', x.group(1)))
        f_outer = lambda x: ast.literal_eval(
            re.sub(pattern, f_inner, x.replace('"', '')).replace('{', '[').replace('}', ']')
        )
        for col in list_cols:
            df[col] = df[col].apply(f_outer)
        print('done!')
    
    return df

## Data Loading
Doing a straight `read_sql` with pandas uses a lot of overhead with the size of our dataset, so we have to use a memory workaround with our custom `read_sql_with_copy` function.

In [4]:
# Example usage of how to get data into
# a DataFrame with the `read_sql_with_copy` function
query = f'''
    SELECT 
        *
    FROM {TABLE_NAME}
    LIMIT 10
 '''

query_df = read_sql_with_copy(
    query = query, 
    db_engine = sqla_conn,
    convert_list_cols = False
)
query_df.head()

Running query...done!


,track_id,song_id,title,track_7digitalid,analysis_sample_rate,audio_md5,danceability,duration,end_of_fade_in,energy,key,key_confidence,loudness,mode,mode_confidence,start_of_fade_out,tempo,time_signature,time_signature_confidence,bars_confidence,bars_start,beats_confidence,beats_start,sections_confidence,sections_start,segments_confidence,segments_loudness_max,segments_loudness_max_time,segments_loudness_start,segments_pitches,segments_start,segments_timbre,tatums_confidence,tatums_start,artist_terms,artist_terms_freq,artist_terms_weight,similar_artists,artist_7digitalid,artist_familiarity,artist_hotttnesss,artist_id,artist_latitude,artist_longitude,artist_location,artist_mbid,artist_name,artist_playmeid,genre,release,release_7digitalid,song_hotttnesss,artist_mbtags,artist_mbtags_count,year
0,TRBGJCT128F933A606,SOBBQDL12A58A7A988,100 Club 1996 ''We Love You Beatles'' - Live,5195440,22050,40f155321c40015d95c7815033a54e15,0,88.73751,0.298,0,2,0.031,-11.229,0,0.331,88.738,190.333,5,0.562,"{0.124,0.112,0.096,0.215,0.054,0.143,0.114,0.1...","{1.05277,2.68331,4.22783,5.88498,7.44629,8.893...","{0.79,0.92,0.696,0.626,0.587,0.519,0.485,0.47,...","{0.04606,0.37526,0.71941,1.05277,1.38778,1.729...","{1.0,0.371,0.166,0.959}","{0.0,35.743,51.07955,83.16827}","{0.0,1.0,0.95,0.212,0.888,0.303,0.343,1.0,0.88...","{-60.0,-14.114,-13.495,-12.97,-13.733,-11.403,...","{0.0,0.14916,0.02785,0.05477,0.08291,0.04679,0...","{-60.0,-60.0,-27.538,-16.692,-25.007,-14.495,-...","{{0.97,1.0,0.502,0.362,0.352,0.377,0.356,0.259...","{0.0,0.29846,0.56594,0.78739,1.04821,1.14653,1...","{{0.0,171.13,9.469,-28.48,57.491,-50.067,14.83...","{0.79,0.92,0.696,0.626,0.587,0.519,0.485,0.47,...","{0.04606,0.37526,0.71941,1.05277,1.38778,1.729...","{punk,italy,""british punk rock"",soundtrack,""pr...","{1.0,0.5596462649523082,0.5100886604134174,0.3...","{1.0,0.7760465835458507,0.7415077362419781,0.4...","{ARNTVM11187B98B4B4,AREKBBQ11F50C4971D,ARNFTGS...",414,0.731184,0.549204,ARD3LXU1187B9ABFC5,NaN,NaN,NaN,e5db18cb-4b1f-496d-a308-548b611090d3,Sex Pistols,1418,NaN,Sex Pistols - The Interviews,467088,NaN,"{british,""punk rock"",uk,london,england,english...","{4,3,3,1,1,1,1,1,1,1}",0
1,TRBGJOG12903CC3E93,SOIZHEV12A8C14138B,Yo Quiero Contigo,7948828,22050,88ff14072530c24e4a71dc36d8cfcd4d,0,167.36608,5.851,0,4,0.049,-8.383,0,0.236,167.366,122.701,5,0.472,"{0.072,0.129,0.13,0.182,0.222,0.037,0.074,0.04...","{1.67692,4.54817,7.04522,9.49961,11.9417,14.38...","{0.349,0.519,0.296,0.501,0.128,0.0,0.427,0.233...","{0.53691,1.10301,1.67692,2.25482,2.82228,3.389...","{1.0,1.0,0.164,0.415,0.501,0.021,0.319,0.447,0...","{0.0,10.4801,42.25348,61.33113,71.11294,94.589...","{1.0,0.171,0.342,0.569,0.868,0.821,0.976,0.502...","{-24.193,-23.662,-26.904,-24.408,-19.328,-26.4...","{0.09915,0.06022,0.07353,0.04847,0.06487,0.044...","{-60.0,-26.43,-30.124,-29.522,-29.05,-37.009,-...","{{0.828,0.564,0.238,0.302,1.0,0.8,0.808,0.586,...","{0.0,0.18848,0.34,0.51361,1.05342,1.25002,1.64...","{{27.46,-55.217,73.516,-97.74,29.328,151.265,7...","{0.256,0.258,0.259,0.26,0.254,0.274,0.271,0.26...","{0.15448,0.35045,0.53691,0.73003,0.91459,1.103...","{reggaeton,""los angeles"",california,""hip hop"",...","{1.0,0.8120981752908923,0.8120981752908923,0.8...","{1.0,0.8843108934201204,0.8139725439930916,0.6...","{AROTGDJ12454A52E29,ARYFNBU12454A4D74D,ARROGBH...",170931,0.610186,0.355320,ARWWPKQ11F4C83D8E2,NaN,NaN,NaN,NaN,Baby Rasta & Gringo,-1,NaN,Sentenciados - Platinum Edition,716669,NaN,{},{},0
2,TRBGJFN128F429AB79,SOHFGYR12A8C13AE07,Vankina vapauden,1170609,22050,77ab2a1f981525fea9c68f3b6958e402,0,189.36118,5.515,0,5,0.142,-10.819,0,0.319,185.481,97.103,3,1.000,"{1.0,0.175,0.103,0.136,0.088,0.243,0.112,0.284...","{1.26612,3.10739,5.00016,6.882,8.75449,10.6566...","{0.0,0.0,0.0,0.0,0.0,0.559,0.74,0.48,0.341,0.3...","{0.64188,1.26612,1.88456,2.47959,3.10739,3.733...","{1.0,1.0,0.397,0.01,0.195,0.517}","{0.0,5.00016,42.05266,54.24716,100.19433,129.2...","{0.0,1.0,0.491,0.446,0.334,0.161,0.75,0.194,0....","{-

## Data Analysis